In [ ]:
# --- Mini-Epoch Test ---

# 1. Set the correction network to train mode (or eval mode if you want no dropout/batchnorm).
correction_net.train()

# 2. Get one batch (sample) from the DataLoader.
data_iter = iter(data_loader)
t_batch, output_batch = next(data_iter)

# If batch_size=1, PyTorch adds a batch dimension.
# Remove or index out that dimension if needed:
t_instance = t_batch.squeeze(0)     # Now shape [num_timepoints]
trajectory_gt = output_batch
trajectory_gt = [state[1:-1] for state in trajectory_gt]     # If 'output_batch' is a list of length 1

tN = t_instance[-1]
psiInitial = output_batch[1]

corrected_traj, solver_t = PRE_hybridSolver(tN,psiInitial, torchsolver.havercampCfun, torchsolver.havercampKfun, torchsolver.havercampthetafun, torchsolver.zeroFun, correction_net)


loss = 0.0
num_steps = len(solver_t)
for j, s in enumerate(solver_t):
    # interpolate_at_time(s, t_instance, trajectory_gt) must be defined by you
    gt_state = interpolate_at_time(s, t_instance, trajectory_gt)
    loss += torch.mean((corrected_traj[j] - gt_state) ** 2)

loss = loss / num_steps

print("Mini-epoch loss (MSE):", loss.item())
